## This notebook aims to develope the Accuracy testing

In [1]:
import numpy as np 
from tools.auxiliary import*
from tools.GH_Quadrature import*
from tools.Ord_Herm_Pol_1 import*
from tools.Num_Stab_Approx import*
from tools.Main_Results import*
import pandas as pd
import math

In [2]:
df =reading("epsi10000.csv")
T  = 10000           #Choose the simulation length for the solution procedure, T<=10,000                    
gam     = 1        # Utility-function parameter
alpha   = 0.36     # Capital share in output
beta    = 0.99     # Discount factor
delta   = 0.02     # Depreciation rate 
rho     = 0.95     # Persistence of the log of the productivity level
sigma   = 0.01    # Standard deviation of shocks to the log of the productivity level
ks = ( (1-beta+beta*delta) / (alpha*beta) )**(1/(alpha-1) )
k = np.array([ks]*(T+1))
a= [1]*(T)
epsi = df.iloc[:,0].astype(float)*sigma
epsi=epsi.tolist()
for i in range(1, T):
    a[i]=a[i-1]**(rho)*math.exp(epsi[i])
a=np.asarray(a)
kdamp = 0.01    
dif_GSSA_1d = 1e+10  
bk_1d  = np.array([0., 0.95, ks*0.05])
bk_1d= np.reshape(bk_1d, (3,1))
k_old = [ks+1]*(T+1)

In [3]:
y = GSSA_main_cycle(T, gam, alpha, beta, delta, kdamp, dif_GSSA_1d, a, bk_1d, k_old, k)

In [4]:
y = y.reshape((y.shape[0],1)) #make sure y is in the right shape
#The GSSA parameters

kdamp = 0.1
dif_GSSA_D = 1e+10

#The matrices of the polynomial coefficients
D_max  = 5 #because of python
npol = np.array([3, 6, 10, 15, 21])

BK = np.zeros((npol[D_max-1], D_max)) #The original paper use this to store results dont need this

# 13. Choose an integration method for computing solutions  
IM  = 10

n_nodes,epsi_nodes, weight_nodes= GH_Quadrature(Qn=10, N=1, vcv=sigma**2)

#make sure to change a into the right shape
a = np.reshape(a, (T, 1))
a1 = np.matmul(np.power(a,rho), np.exp(epsi_nodes.transpose()))

#14. Choose a regression specification 
RM = 6           # Choose a regression method: 
                 # 1=OLS,          2=LS-SVD,   3=LAD-PP,  4=LAD-DP, 
                 # 5=RLS-Tikhonov, 6=RLS-TSVD, 7=RLAD-PP, 8=RLAD-DP
normalize = 1    # Option of normalizing the data; 0=unnormalized data; 
                 # 1=normalized data                    
penalty = 7      # Degree of regularization for a regularization methods, 
                 # RM=5,6,7,8 (must be negative, e.g., -7 for RM=5,7,8 
                 # and must be positive, e.g., 7, for RM=6)
PF = 0           # Choose a polynomial family; 0=Ordinary (default);  
                 # 1=Hermite
# 15. Initialize the capital series
zb = np.matrix([[np.mean(k[0:T]), np.mean(a[0:T])], [np.std(k[0:T]), np.std(a[0:T])]])
z = np.concatenate((k[0:T].reshape(T,1), a[0:T].reshape(T,1)), axis=1)
k_old = [ks+1]*(T+1)

In [5]:
BK = []
for d in range(1, D_max+1):
    BK.append(GSSA_poly(T, a, z, d, PF, zb, RM, penalty, normalize, dif_GSSA_D, kdamp, alpha, beta, delta, k, gam, y, k_old, a1, IM, n_nodes, weight_nodes, checker= 0))

In [28]:
BK1DF = pd.DataFrame(BK[0].tolist())
BK2DF = pd.DataFrame(BK[1].tolist())
BK3DF = pd.DataFrame(BK[2].tolist())
BK4DF = pd.DataFrame(BK[3].tolist())
BK5DF = pd.DataFrame(BK[4].tolist())
yDF = pd.DataFrame(y.tolist())

In [30]:
BK1DF.to_csv("Accuracy_BK1.csv", index=False)
BK2DF.to_csv("Accuracy_BK2.csv", index=False)
BK3DF.to_csv("Accuracy_BK3.csv", index=False)
BK4DF.to_csv("Accuracy_BK4.csv", index=False)
BK5DF.to_csv("Accuracy_BK5.csv", index=False)
yDF.to_csv("Accuracy_y.csv", index=False) #since the calculation is super long 

In [25]:
BK[0]

array([[-1.76699623],
       [ 0.97049037],
       [ 3.19294961]])

In [27]:
BK[0].shape

(3, 1)

## Accuracy:

In [ ]:
#need to make sure to change the df into a with an already given shape

In [2]:
BK1 = pd.read_csv("Accuracy_BK1.csv").to_numpy()

Prerequisite:

In [3]:
df =reading("epsi10000.csv")
T  = 10000           #Choose the simulation length for the solution procedure, T<=10,000                    
gam     = 1        # Utility-function parameter
alpha   = 0.36     # Capital share in output
beta    = 0.99     # Discount factor
delta   = 0.02     # Depreciation rate 
rho     = 0.95     # Persistence of the log of the productivity level
sigma   = 0.01    # Standard deviation of shocks to the log of the productivity level
ks = ( (1-beta+beta*delta) / (alpha*beta) )**(1/(alpha-1) )
k = np.array([ks]*(T+1))
a= [1]*(T)
epsi = df.iloc[:,0].astype(float)*sigma
epsi=epsi.tolist()
for i in range(1, T):
    a[i]=a[i-1]**(rho)*math.exp(epsi[i])
a=np.asarray(a)


#The matrices of the polynomial coefficients
D_max  = 5 #because of python
npol = np.array([3, 6, 10, 15, 21])

# 13. Choose an integration method for computing solutions  
IM  = 10



#14. Choose a regression specification 
RM = 6           # Choose a regression method: 
                 # 1=OLS,          2=LS-SVD,   3=LAD-PP,  4=LAD-DP, 
                 # 5=RLS-Tikhonov, 6=RLS-TSVD, 7=RLAD-PP, 8=RLAD-DP
PF = 0           # Choose a polynomial family; 0=Ordinary (default);  
                 # 1=Hermite
# 15. Initialize the capital series
zb = np.matrix([[np.mean(k[0:T]), np.mean(a[0:T])], [np.std(k[0:T]), np.std(a[0:T])]])


In [6]:



T_test = 10200

df =reading("epsi_test.csv")
epsi_test = sigma*df.to_numpy().astype(float)
a_test = [1]
for i in range(1,T_test):
    value = a_test[i-1]**(rho)*math.exp(float(epsi_test[i]))
    a_test.append(value)

IM_test = 10

k_test = [ks]
for d in range(1, D_max+1):
    #refressing k_test to make sure that k_test is always 10200
    #k_test = [ks]
    for i in range(T_test):
        X_test = Ord_Herm_Pol_1(np.array([k_test[i], a_test[i]]).reshape([1,2]),d,PF,zb) # D = 1 for now, we will plug this in another for loop
        value = float(np.matmul(X_test, BK[d-1]))
        k_test.append(value)
        #unfinished function
        # testing it below
discard = 200 #new defined value


Testing area:

In [4]:
T_test = 10200
df =reading("epsi10000.csv")
T  = 10000           #Choose the simulation length for the solution procedure, T<=10,000                    
gam     = 1        # Utility-function parameter
alpha   = 0.36     # Capital share in output
beta    = 0.99     # Discount factor
delta   = 0.02     # Depreciation rate 
rho     = 0.95     # Persistence of the log of the productivity level
sigma   = 0.01    # Standard deviation of shocks to the log of the productivity level
ks = ( (1-beta+beta*delta) / (alpha*beta) )**(1/(alpha-1) )
k = np.array([ks]*(T+1))
a= [1]*(T)
epsi = df.iloc[:,0].astype(float)*sigma
epsi=epsi.tolist()
for i in range(1, T):
    a[i]=a[i-1]**(rho)*math.exp(epsi[i])
a=np.asarray(a)
PF = 0
zb = np.matrix([[np.mean(k[0:T]), np.mean(a[0:T])], [np.std(k[0:T]), np.std(a[0:T])]])


df =reading("epsi_test.csv")
epsi_test = sigma*df.to_numpy().astype(float)
a_test = [1]
for i in range(1,T_test):
    value = a_test[i-1]**(rho)*math.exp(float(epsi_test[i]))
    a_test.append(value)

IM_test = 10

k_test = [ks]

In [5]:
d = 1
for i in range(T_test):
        X_test = Ord_Herm_Pol_1(np.array([k_test[i], a_test[i]]).reshape([1,2]),d,PF,zb) # D = 1 for now, we will plug this in another for loop
        value = float(np.matmul(X_test, BK1))
        k_test.append(value)

In [ ]:
def Accuracy_Test_1(sigma,rho,beta,gam,alpha,delta,k,a,bk,D,IM,PF,zb,discard):
    '''
    '''
    n_nodes,epsi_nodes, weight_nodes = GH_Quadrature(Qn=IM, N=1, vcv=sigma**2)

    for i in range(a):
        k0 = k[i]
        a0 = a[i,0]
        k1 = np.matmul(Ord_Herm_Pol_1(np.array([k[i],float(a[i])]).reshape((1,2)), D=d, PF=PF, zb=zb), bk)
        c0 = k0

In [11]:
for i in range(a):
    i = 1
    k0 = float(k[i])
    a0 = float(a[i,0])
    k1 = float(np.matmul(Ord_Herm_Pol_1(np.array([k0,a0]).reshape((1,2)), D, PF=PF, zb=zb), bk))
    c0 = k0**alpha*a0 - k1+ (1-delta)*k0
    a1 = a0**rho*np.exp(epsi_nodes)
    k1_dup1 = k1*np.ones((n_nodes,1))
    x1 = Ord_Herm_Pol_1(np.concatenate((k1_dupl1, a1), axis=1),D,PF,zb)
    k2 = np.matmul(x1, bk)
    c1 = k1**alpha*a1 - k2 + (1-delta)*k1
    errors = 


1.0

In [33]:
    i = 1
    k0 = float(k[i])
    a0 = float(a[i,0])
    k1 = float(np.matmul(Ord_Herm_Pol_1(np.array([k0,a0]).reshape((1,2)), D=1, PF=PF, zb=zb), BK[0]))
    c0 = k0**alpha*a0 - k1+ (1-delta)*k0
    a1 = a0**rho*np.exp(epsi_nodes)
    k1_dupl1 = k1*np.ones((n_nodes,1))
    x1 = Ord_Herm_Pol_1(np.concatenate((k1_dupl1, a1), axis=1), 1, PF, zb)
    k2 = np.matmul(x1, BK[0])
    c1 = k1**alpha*a1 - k2 + (1-delta)*k1
    errors = 


In [29]:
np.concatenate((k1_dupl1, a1), axis=1)

NameError: name 'k1_dupl1' is not defined

In [23]:
 c1 = k1**alpha*a1 - k2 + (1-delta)*k1

NameError: name 'k1' is not defined

In [22]:
a1 = a0**rho*np.exp(epsi_nodes)

In [17]:
n_nodes,epsi_nodes, weight_nodes = GH_Quadrature(Qn=IM, N=1, vcv=sigma**2)

In [ ]:
        X1 = Ord_Herm_Pol_1([k1_dupl a1(:,1)],D,PF,zb);
        % Form a complete polynomial of degree D (at t+1) in the given point 
        
        k2(:,1) =  X1*bk;
        % Compute capital of period t+2 (chosen at t+1) using the fifth-
        % degree capital policy function; n_nodes-by-1 
        
        c1(:,1) =  k1(1,1)^alpha*a1(:,1) - k2(:,1) + (1-delta)*k1(1,1);